In [1]:
import pandas as pd

pd.set_option('future.no_silent_downcasting', True)

---
# Create NEW, CLEAN Dataset
---

## Construct IMDB sub-dataset

### Read data from csv files into data frames and perform necessary transformations

In [2]:
# read in imdb titles data
imdb_titles_df = pd.read_csv(
    filepath_or_buffer='../data/imdbtitles.tsv',
    sep='\t',
    low_memory=False
)

# convert startYear to numeric to allow for joining with netflix data
imdb_titles_df['startYear'] = pd.to_numeric(
    imdb_titles_df['startYear'],
    errors='coerce'
)

# read in imdb ratings data
imdb_ratings_df = pd.read_csv(
    filepath_or_buffer='../data/imdbratings.tsv',
    sep='\t',
    low_memory=False
)

### Separate imdb data frame into two: movies, tv shows

In [3]:
# First we'll create our two 'imdb_titles' data frames
imdb_titles_mv_df = imdb_titles_df.loc[imdb_titles_df['titleType'] == 'movie']
imdb_titles_tv_df = imdb_titles_df.loc[imdb_titles_df['titleType'] == 'tvSeries']
# free up memory
del imdb_titles_df

# Next we'll create our two 'imdb_ratings' data frames
imdb_ratings_mv_df = imdb_ratings_df.loc[imdb_ratings_df['tconst'].isin(imdb_titles_mv_df['tconst'])]
imdb_ratings_tv_df = imdb_ratings_df.loc[imdb_ratings_df['tconst'].isin(imdb_titles_tv_df['tconst'])]
# free up memory
del imdb_ratings_df

# Now we'll merge the two 'imdb_titles' data frames with the two 'imdb_ratings' data frames
imdb_mv_df = pd.merge(
    left=imdb_titles_mv_df,
    right=imdb_ratings_mv_df,
    how='inner',
    on='tconst'
).sort_values(
    by=['primaryTitle', 'numVotes'],
    ascending=False
    ).drop_duplicates(
        subset=[
            'primaryTitle',
            'startYear'
            ],
        keep='first',
        ignore_index=True
            )

# free up memory
del imdb_titles_mv_df, imdb_ratings_mv_df

imdb_tv_df = pd.merge(
    left=imdb_titles_tv_df,
    right=imdb_ratings_tv_df,
    how='inner',
    on='tconst'
).sort_values(
    by=['primaryTitle', 'numVotes'],
    ascending=False
    ).drop_duplicates(
        subset=[
            'primaryTitle',
            'startYear'],
            keep='first',
            ignore_index=True
            )

# free up memory
del imdb_titles_tv_df, imdb_ratings_tv_df

## Construct Netflix sub-dataset

### read netflix data from csv files into data frames and perform necessary transormations

In [4]:
# read in netflix_titles data
netflix_titles_df = pd.read_csv(
    filepath_or_buffer='../data/netflix_titles.csv',
    low_memory=False
)

# read in netflixdump data
netflix_dump_df = pd.read_csv(
    filepath_or_buffer='../data/netflixdump.csv',
    low_memory=False
)

# create a copy of the netflix_dump_df to illustrate a potential roadbump, momentarily.
netflix_dump_df_copy = netflix_dump_df.copy()

# Create a list of all the titles in the 'title' column of the netflix_dump_df with both an English and non-English title
multi_lang_titles = [(title[0], title[1]) for title in netflix_dump_df['title'].map(lambda x: x.split('//')) if len(title) > 1]


# split 'title' column on '//' and keep only the half in front of the '//' if '//' is present because it is in English
netflix_dump_df['title'] = netflix_dump_df['title'].map(lambda x: x.split('//')[0].strip() if '//' in x else x)

### Let's check for duplicates in the kaggle netflix data.

In [6]:
netflix_titles_df.title.duplicated().sum()

0

## Without removing the separator between the English title characters and those in another language, we do not see any title duplicates in the Netflix data dump.

#### TOTAL LIST OF TITLES BEFORE TRANSFORMATION

In [7]:
netflix_dump_df_copy.title

0                  The Night Agent: Season 1
1                  Ginny & Georgia: Season 2
2         The Glory: Season 1 // 더 글로리: 시즌 1
3                        Wednesday: Season 1
4        Queen Charlotte: A Bridgerton Story
                        ...                 
18209                              راس السنة
18210                           心が叫びたがってるんだ。
18211                              두근두근 내 인생
18212                                 라디오 스타
18213                                 선생 김봉두
Name: title, Length: 18214, dtype: object

#### Using value_counts(), we can verify the lack of duplicate titles prior to transformation.

In [8]:
netflix_dump_df_copy.title.value_counts()

title
The Night Agent: Season 1                        1
Romanzo Criminale (2005)                         1
Russell Howard: Recalibrate                      1
RuPaul's Secret Celebrity Drag Race: Season 1    1
Rumble: The Indians Who Rocked the World         1
                                                ..
Hero // 영웅                                       1
His House                                        1
Hostel: Part III                                 1
How to Be a Latin Lover                          1
선생 김봉두                                           1
Name: count, Length: 18214, dtype: int64

#### TOTAL SUM OF DUPLICATED TITLES PRIOR TO TRANSFORMATION

In [9]:
netflix_dump_df_copy.title.duplicated().sum()

0

<hr />

#### Here is a list of all the titles from the initial Netflix dump that contain both an English and a non-English title.

In [10]:
multi_lang_titles

[('The Glory: Season 1 ', ' 더 글로리: 시즌 1'),
 ('Physical: 100: Season 1 ', ' 피지컬: 100: 시즌 1'),
 ('Crash Course in Romance: Limited Series ', ' 일타 스캔들: 리미티드 시리즈'),
 ('Fake Profile: Season 1 ', ' Perfil falso: Temporada 1'),
 ('Doctor Cha: Limited Series ', ' 닥터 차정숙: 리미티드 시리즈'),
 ('The Marked Heart: Season 2 ', ' Pálpito: Temporada 2'),
 ('The Surrogacy: Season 1 ', ' Madre de alquiler: Temporada 1'),
 ('Alice in Borderland: Season 2 ', ' 今際の国のアリス: シーズン2'),
 ('Alchemy of Souls: Part 1 ', ' 환혼: 파트 1'),
 ('Til Money Do Us Part: Season 1 ',
  ' Hasta que la plata nos separe: Temporada 1'),
 ('Mr. Queen ', ' 철인왕후'),
 ('Alchemy of Souls: Part 2 ', ' 환혼: 파트 2'),
 ('The Good Bad Mother: Limited Series ', ' 나쁜엄마: 리미티드 시리즈'),
 ('Bloodhounds: Season 1 ', ' 사냥개들: 시즌 1'),
 ('Wrong Side of the Tracks: Season 2 ', ' Entrevías: Temporada 2'),
 ('Triptych: Season 1 ', ' Tríada: Temporada 1'),
 ('Extraordinary Attorney Woo: Season 1 ', ' 이상한 변호사 우영우: 시즌 1'),
 ('The Snow Girl: Season 1 ', ' La chica de niev

<hr />

## Once the separating characters are removed, we are left with numerous duplicates.

#### TOTAL NUMBER OF DUPLICATE TITLES AFTER TRANFORMATION

In [11]:
netflix_dump_df.title.duplicated().sum()

53

#### Again employing the value_counts() method, we can easily see the duplicates we are left with.

In [12]:
netflix_dump_df.title.value_counts().head(53)

title
Perfect Strangers                                4
Cargo                                            3
Switch                                           3
Noise                                            3
First Love                                       2
Way Back Home                                    2
Man in Love                                      2
The Wave                                         2
Yu-Gi-Oh!: Season 4                              2
Aftermath                                        2
Tokyo Ghoul                                      2
A Beautiful Life                                 2
The Wedding                                      2
The Club: Season 1                               2
We Can Be Heroes                                 2
Shadow: Season 1                                 2
No Mercy                                         2
Portrait of a Beauty                             2
Signal: Season 1                                 2
Unrequited Love: Season 1

### How many titles still match between the two Netflix sources after performing the transformation?

In [13]:
print(len(list(set(netflix_dump_df['title'].to_list()) & set(netflix_titles_df['title'].to_list()))))

2903


### Let's create a data frame from the subset of Netflix data whose titles match across the two Netflix sources after removing the non-English characters.

In [14]:
abridged_netflix_df = netflix_titles_df.loc[netflix_titles_df['title'].isin(netflix_dump_df['title'])].copy()
abridged_netflix_df.sample(10)

# free up memory
del netflix_titles_df

#### Are there duplicates still remaining?

In [15]:
abridged_netflix_df.title.value_counts()

title
My Little Pony: A New Generation    1
Carbon                              1
Once Again                          1
Sisters                             1
The Laws of Thermodynamics          1
                                   ..
The Hater                           1
The Speed Cubers                    1
Double World                        1
Animal Crackers                     1
Zombieland                          1
Name: count, Length: 2903, dtype: int64

<hr />

## We'll use the abridged Netflix data we created when we joined the two Netflix sets on matching tiles after removing the non-English titles.

In [17]:
# First we'll create our two 'netflix_titles' data frames
netflix_titles_mv_df = abridged_netflix_df.loc[abridged_netflix_df['type'] == 'Movie'].copy()
netflix_titles_tv_df = abridged_netflix_df.loc[abridged_netflix_df['type'] == 'TV Show'].copy()
# free up memory
del abridged_netflix_df

# Now we'll merge the two 'abridged' data frames with the 'netflix_dump' data frame, one at a time, joining on 'title'
netflix_mv_df = pd.merge(
    left=netflix_titles_mv_df,
    right=netflix_dump_df,
    how='inner',
    on='title'
)

netflix_tv_df = pd.merge(
    left=netflix_titles_tv_df,
    right=netflix_dump_df,
    how='inner',
    on='title'
)

# free up memory
del netflix_titles_mv_df, netflix_titles_tv_df, netflix_dump_df

## FINAL NETFLIX DATA FRAMES INSPECTION

#### MOVIES

In [18]:
netflix_mv_df.info(
    memory_usage='deep'
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827 entries, 0 to 2826
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   show_id              2827 non-null   object
 1   type                 2827 non-null   object
 2   title                2827 non-null   object
 3   director             2748 non-null   object
 4   cast                 2665 non-null   object
 5   country              2599 non-null   object
 6   date_added           2827 non-null   object
 7   release_year         2827 non-null   int64 
 8   rating               2827 non-null   object
 9   duration             2824 non-null   object
 10  listed_in            2827 non-null   object
 11  description          2827 non-null   object
 12  available globally?  2827 non-null   object
 13  release date         1167 non-null   object
 14  hours viewed         2827 non-null   object
dtypes: int64(1), object(14)
memory usage: 3.0 MB


In [19]:
netflix_tv_df.info(
    memory_usage='deep'
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   show_id              93 non-null     object
 1   type                 93 non-null     object
 2   title                93 non-null     object
 3   director             11 non-null     object
 4   cast                 88 non-null     object
 5   country              72 non-null     object
 6   date_added           93 non-null     object
 7   release_year         93 non-null     int64 
 8   rating               93 non-null     object
 9   duration             93 non-null     object
 10  listed_in            93 non-null     object
 11  description          93 non-null     object
 12  available globally?  93 non-null     object
 13  release date         17 non-null     object
 14  hours viewed         93 non-null     object
dtypes: int64(1), object(14)
memory usage: 97.6 KB


### It appears we need to recast the 'hours viewed' column as int32.  We'll wait to recast the 'type', 'rating' and 'available globally?' columns as 'category' until after we've merged the dataframes to avoid category conflicts.

In [20]:
# We'll start with the movies data frame, first removing the commas from the 'hours viewed' column.
netflix_mv_df['hours viewed'] = netflix_mv_df['hours viewed'].map(lambda x: x.replace(',', '') if ',' in x else x)

netflix_mv_df = netflix_mv_df.astype(
    {
        'release_year': 'int32',
        'hours viewed': 'int32'
    }
)

# Now we'll do the same for the tv shows data frame.
netflix_tv_df['hours viewed'] = netflix_tv_df['hours viewed'].map(lambda x: x.replace(',', '') if ',' in x else x)

netflix_tv_df = netflix_tv_df.astype(
    {
        'release_year': 'int32',
        'hours viewed': 'int32'
    }
)

### Let's give the Netflix data a last check before we merge

In [21]:
print(netflix_mv_df.info(
    memory_usage='deep'
), netflix_tv_df.info(
    memory_usage='deep'
))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827 entries, 0 to 2826
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   show_id              2827 non-null   object
 1   type                 2827 non-null   object
 2   title                2827 non-null   object
 3   director             2748 non-null   object
 4   cast                 2665 non-null   object
 5   country              2599 non-null   object
 6   date_added           2827 non-null   object
 7   release_year         2827 non-null   int32 
 8   rating               2827 non-null   object
 9   duration             2824 non-null   object
 10  listed_in            2827 non-null   object
 11  description          2827 non-null   object
 12  available globally?  2827 non-null   object
 13  release date         1167 non-null   object
 14  hours viewed         2827 non-null   int32 
dtypes: int32(2), object(13)
memory usage: 2.8 MB
<class 'pa

In [20]:
# Now we'll take a quick look at the descriptive statistics for the two data frames.
print(netflix_mv_df.describe(
    include='all'
).T, netflix_tv_df.describe(
    include='all'
).T)

                      count unique  \
show_id                2827   2811   
type                   2827      1   
title                  2827   2811   
director               2748   2109   
cast                   2665   2540   
country                2599    362   
date_added             2827   1080   
release_year         2827.0    NaN   
rating                 2827     16   
duration               2824    180   
listed_in              2827    208   
description            2827   2803   
available globally?    2827      2   
release date           1167    708   
hours viewed         2827.0    NaN   

                                                                   top  freq  \
show_id                                                          s2031     3   
type                                                             Movie  2827   
title                                                            Cargo     3   
director                                                  Marcus Raboy 

##### Check for any remaining duplicates.

In [22]:
print(netflix_mv_df.title.value_counts(),
      netflix_tv_df.title.value_counts())

title
Cargo                     3
We Can Be Heroes          2
The Killer                2
Manhunt                   2
The King                  2
                         ..
Offering to the Storm     1
Romance Doll              1
The Kissing Booth 2       1
The Larva Island Movie    1
Zombieland                1
Name: count, Length: 2811, dtype: int64 title
Wolf                                        2
The Circle                                  1
Bad Blood                                   1
The Haunting of Hill House                  1
Mystery Science Theater 3000: The Return    1
                                           ..
Barbarians                                  1
Bakugan: Battle Planet                      1
Bakugan: Armored Alliance                   1
Detention                                   1
The Hunt                                    1
Name: count, Length: 92, dtype: int64


### Let's drop any duplicates for now.  We have enough data to move on, for now.

In [23]:
netflix_mv_df = netflix_mv_df.loc[~(netflix_mv_df['title'].duplicated(keep=False)), :]
netflix_tv_df = netflix_tv_df.loc[~(netflix_tv_df['title'].duplicated(keep=False)), :]

print(netflix_mv_df.title.value_counts(),
      netflix_tv_df.title.value_counts())

title
My Little Pony: A New Generation                   1
Zion                                               1
Mersal                                             1
Pad Man                                            1
Toilet: Ek Prem Katha                              1
                                                  ..
Dragons: Rescue Riders: Secrets of the Songwing    1
Nimbe                                              1
Offering to the Storm                              1
Romance Doll                                       1
Zombieland                                         1
Name: count, Length: 2796, dtype: int64 title
The Circle                                  1
Sisters                                     1
The Haunting of Hill House                  1
Mystery Science Theater 3000: The Return    1
Flowers                                     1
                                           ..
Barbarians                                  1
Bakugan: Battle Planet                     

## Merge the Netflix and IMDB datasets

### Merge 'movie' data frames

In [24]:
mv_df = pd.merge(
    left=imdb_mv_df,
    right=netflix_mv_df,
    how='inner',
    left_on=['primaryTitle', 'startYear'],
    right_on=['title', 'release_year']
).replace(
    to_replace='\\N',
    value=0
).sort_values(
    by=['primaryTitle', 'numVotes'],
    ascending=False
).drop_duplicates(
    subset=[
        'primaryTitle',
        'startYear'
    ],
    keep='first',
    ignore_index=True
)

### Merge 'tv' data frames

In [25]:
tv_df = pd.merge(
    left=imdb_tv_df,
    right=netflix_tv_df,
    how='inner',
    left_on=['primaryTitle', 'startYear'],
    right_on=['title', 'release_year']
).replace(
    to_replace='\\N',
    value=0
).sort_values(
    by=['primaryTitle', 'numVotes'],
    ascending=False
).drop_duplicates(
    subset=[
        'primaryTitle',
        'startYear'
    ],
    keep='first',
    ignore_index=True
)

### Finally, concatenate the two separate data frames into a single data source

In [26]:
# stack the two data frames on top of each other
df = pd.concat(
    [
        mv_df,
        tv_df
    ]
).rename({
    'show_id': 'nfShowId',
    'date_added': 'dateAdded',
    'release_year': 'releaseYear',
    'listed_in': 'listedIn',
    'available globally?': 'availableGlobally',
    'hours viewed': 'hoursViewed'},
axis=1
).astype({
    'titleType': 'category',
    'isAdult': 'category',
    'startYear': 'int32',
    'runtimeMinutes': 'int32',
    'genres': 'category',
    'averageRating': 'float64',
    'numVotes': 'int32',
    'type': 'category',
    'country': 'category',
    'rating': 'category',
    'duration': 'category',
    'listedIn': 'category',
    'availableGlobally': 'category',
    'hoursViewed': 'int32'
}
)

# free up memory
del mv_df, tv_df

# send all text to lower case
df = df.map(lambda x: x.lower() if type(x) == str else x)

df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,...,country,dateAdded,releaseYear,rating,duration,listedIn,description,availableGlobally,release date,hoursViewed
0,tt6213294,movie,"¡ay, mi madre!","¡ay, mi madre!",0,2019,0,84,comedy,3.9,...,spain,"july 19, 2019",2019,tv-ma,81 min,"comedies, international movies","when her estranged mother suddenly dies, a wom...",yes,NaN,300000
1,tt1156398,movie,zombieland,zombieland,0,2009,0,88,"action,comedy,horror",7.6,...,united states,"november 1, 2019",2009,r,88 min,"comedies, horror movies",looking to survive in a world taken over by zo...,no,NaN,10500000
2,tt1562872,movie,zindagi na milegi dobara,zindagi na milegi dobara,0,2011,0,155,"comedy,drama,musical",8.2,...,india,"december 15, 2019",2011,tv-14,154 min,"comedies, dramas, international movies",three friends on an adventurous road trip/bach...,no,NaN,4100000
3,tt1007028,movie,zack and miri make a porno,zack and miri make a porno,0,2008,0,101,"comedy,romance",6.5,...,united states,"may 1, 2021",2008,r,101 min,"comedies, independent movies, romantic movies",zack and miri make and star in an adult film t...,no,NaN,100000
4,tt0382383,movie,yuva,yuva,0,2004,0,161,"action,drama,thriller",7.3,...,india,"march 1, 2018",2004,tv-14,160 min,"dramas, international movies",an accident on a bridge ties together three yo...,no,NaN,200000


In [27]:
df.info(
    verbose=True,
    memory_usage='deep'
)

<class 'pandas.core.frame.DataFrame'>
Index: 1843 entries, 0 to 34
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   tconst             1843 non-null   object  
 1   titleType          1843 non-null   category
 2   primaryTitle       1843 non-null   object  
 3   originalTitle      1843 non-null   object  
 4   isAdult            1843 non-null   category
 5   startYear          1843 non-null   int64   
 6   endYear            1843 non-null   object  
 7   runtimeMinutes     1843 non-null   int64   
 8   genres             1843 non-null   category
 9   averageRating      1843 non-null   float64 
 10  numVotes           1843 non-null   int64   
 11  nfShowId           1843 non-null   object  
 12  type               1843 non-null   category
 13  title              1843 non-null   object  
 14  director           1790 non-null   object  
 15  cast               1755 non-null   object  
 16  country      

In [28]:
df.describe(
    include='all'
).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
tconst,1843,1843,tt6213294,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
titleType,1843,2,movie,1808,NaN,NaN,NaN,NaN,NaN,NaN,NaN
primaryTitle,1843,1843,"¡ay, mi madre!",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
originalTitle,1843,1843,"¡ay, mi madre!",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
isAdult,1843,1,0,1843,NaN,NaN,NaN,NaN,NaN,NaN,NaN
startYear,1843.0,NaN,NaN,NaN,2014.170374,8.357138,1954.0,2012.5,2017.0,2019.0,2021.0
endYear,1843.0,10.0,0.0,1820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
runtimeMinutes,1843.0,NaN,NaN,NaN,108.468801,25.119291,0.0,94.0,105.0,120.5,280.0
genres,1843,265,drama,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
averageRating,1843.0,NaN,NaN,NaN,6.274173,1.062583,1.9,5.6,6.3,7.0,9.3


---
# Output New Optimized Dataset
---

In [29]:
df.to_csv(
    path_or_buf='../data/FULLY_COMBINED_DATASET.csv',
    index=False
)

# free up memory
del df

print('done')

done
